In [ ]:
import os
MacOSPath = "/Users/grantseiter/Dropbox/github/Tax-Benefits-Of-Parenthood/Tax-Calculator-3.0.0/"
MsWinPath = "C:\\Users\\grant.seiter\\Dropbox\\github\\Tax-Benefits-Of-Parenthood\\Tax-Calculator-3.0.0\\"
os.chdir(MacOSPath)
workdir = os.getcwd()
print (workdir)
from taxcalc import *
from behresp import *
import pandas as pd
import numpy as np

In [ ]:
# Read JSON Reforms
reform0 = Policy.read_json_reform('../reforms/ctc/zero-reform.json')
reformClinton = Policy.read_json_reform('../reforms/ctc/1997-1999-reform.json')
reformBush1 = Policy.read_json_reform('../reforms/ctc/2001-2002-reform.json')
reformBush2 = Policy.read_json_reform('../reforms/ctc/2003-reform.json')
reformBush3 = Policy.read_json_reform('../reforms/ctc/2004-2007-reform.json')
reform2008 = Policy.read_json_reform('../reforms/ctc/2008-reform.json')
reformOB = Policy.read_json_reform('../reforms/ctc/2009-2017-reform.json')
reformTCJA = Policy.read_json_reform('../reforms/ctc/2018-2025-reform.json')

In [ ]:
# Combined Tax by Liab

def combined(refchoice, basechoice, baselinelaw, segmentchoice, subchoice, cgchoice):
    rec = Records()
    refs_policy = refchoice
    refs_base = basechoice
    df = pd.DataFrame(index=["Combined Tax Liability"
                         ])

    for year in range(2021,2022):
        pol_base = Policy()
        pol_base.implement_reform(baselinelaw)
        pol_ref = Policy()
        pol_ref.implement_reform(baselinelaw)
        col=[]
        response_elasticities = {'sub': subchoice, 'cg': cgchoice}
        for (ref_policy, ref_base) in zip(refs_policy, refs_base):
            pol_base.implement_reform(ref_base)
            calc_base=Calculator(pol_base, rec)
            calc_base.advance_to_year(year)
            pol_ref.implement_reform(ref_policy)
            calc_ref=Calculator(pol_ref, rec)
            calc_ref.advance_to_year(year)
            calc_ref.calc_all()
            base_df, ref_df = response(calc_base, calc_ref, response_elasticities)

            df1 = copy.deepcopy(base_df)
            df1 = df1.assign(ref_combined = ref_df['combined'])
            df1 = df1.assign(ref_iitax = ref_df['iitax'])

            dfp = df1[df1['iitax']>0]
            df0 = df1[df1['iitax']==0]
            dfn = df1[df1['iitax']<0]

            dfpp = dfp[dfp['ref_iitax']>0]
            dfp0 = dfp[dfp['ref_iitax']==0]
            dfpn = dfp[dfp['ref_iitax']<0]

            df0p = df0[df0['ref_iitax']>0]
            df00 = df0[df0['ref_iitax']==0]
            df0n = df0[df0['ref_iitax']<0]

            dfnp = dfn[dfn['ref_iitax']>0]
            dfn0 = dfn[dfn['ref_iitax']==0]
            dfnn = dfn[dfn['ref_iitax']<0]
            
            if segmentchoice=='pp':
                base_rev = (dfpp['combined'] * dfpp['s006']).sum()/1e9
                ref_rev = (dfpp['ref_combined'] * dfpp['s006']).sum()/1e9
            if segmentchoice=='p0':
                base_rev = (dfp0['combined'] * dfp0['s006']).sum()/1e9
                ref_rev = (dfp0['ref_combined'] * dfp0['s006']).sum()/1e9
            if segmentchoice=='pn':
                base_rev = (dfpn['combined'] * dfpn['s006']).sum()/1e9
                ref_rev = (dfpn['ref_combined'] * dfpn['s006']).sum()/1e9
            if segmentchoice=='0p':
                base_rev = (df0p['combined'] * df0p['s006']).sum()/1e9
                ref_rev = (df0p['ref_combined'] * df0p['s006']).sum()/1e9
            if segmentchoice=='00':
                base_rev = (df00['combined'] * df00['s006']).sum()/1e9
                ref_rev = (df00['ref_combined'] * df00['s006']).sum()/1e9
            if segmentchoice=='0n':
                base_rev = (df0n['combined'] * df0n['s006']).sum()/1e9
                ref_rev = (df0n['ref_combined'] * df0n['s006']).sum()/1e9
            if segmentchoice=='np':
                base_rev = (dfnp['combined'] * dfnp['s006']).sum()/1e9
                ref_rev = (dfnp['ref_combined'] * dfnp['s006']).sum()/1e9
            if segmentchoice=='n0':
                base_rev = (dfn0['combined'] * dfn0['s006']).sum()/1e9
                ref_rev = (dfn0['ref_combined'] * dfn0['s006']).sum()/1e9
            if segmentchoice=='nn':
                base_rev = (dfnn['combined'] * dfnn['s006']).sum()/1e9
                ref_rev = (dfnn['ref_combined'] * dfnn['s006']).sum()/1e9
            if segmentchoice=='total':
                base_rev= (base_df['combined'] * base_df['s006']).sum()/1e9
                ref_rev = (ref_df['combined'] * ref_df['s006']).sum()/1e9
                
            dif_rev = ref_rev - base_rev
            col.append(round(dif_rev,2))
        df[year] = col

    # Revenue Table
    table = df
    return table

In [ ]:
# COMBINED by Liab w/ Beh Resp 

def beh_resp(refchoice, basechoice, baselinelaw, segmentchoice, subchoice, cgchoice):
    rec = Records()
    refs_policy = refchoice
    refs_base = basechoice
    df = pd.DataFrame(index=["Combined Tax Liability"
                         ])
    
    for year in range(2021,2022):
        pol_base = Policy()
        pol_base.implement_reform(baselinelaw)
        pol_ref = Policy()
        pol_ref.implement_reform(baselinelaw)
        col=[]
        response_elasticities = {'sub': subchoice, 'cg': cgchoice}
        for (ref_policy, ref_base) in zip(refs_policy, refs_base):
            pol_base.implement_reform(ref_base)
            calc_base=Calculator(pol_base, rec)
            calc_base.advance_to_year(year)
            pol_ref.implement_reform(ref_policy)
            calc_ref=Calculator(pol_ref, rec)
            calc_ref.advance_to_year(year)
            calc_ref.calc_all()
            base_df, ref_df = response(calc_base, calc_ref, response_elasticities)

            df1 = copy.deepcopy(base_df)
            df1 = df1.assign(ref_combined = ref_df['combined'])
            df1 = df1.assign(ref_iitax = ref_df['iitax'])

            dfp = df1[df1['iitax']>0]
            df0 = df1[df1['iitax']==0]
            dfn = df1[df1['iitax']<0]

            dfpp = dfp[dfp['ref_iitax']>0]
            dfp0 = dfp[dfp['ref_iitax']==0]
            dfpn = dfp[dfp['ref_iitax']<0]

            df0p = df0[df0['ref_iitax']>0]
            df00 = df0[df0['ref_iitax']==0]
            df0n = df0[df0['ref_iitax']<0]

            dfnp = dfn[dfn['ref_iitax']>0]
            dfn0 = dfn[dfn['ref_iitax']==0]
            dfnn = dfn[dfn['ref_iitax']<0]
            
            if segmentchoice=='pp':
                base_rev = (dfpp['combined'] * dfpp['s006']).sum()/1e9
                ref_rev = (dfpp['ref_combined'] * dfpp['s006']).sum()/1e9
            if segmentchoice=='p0':
                base_rev = (dfp0['combined'] * dfp0['s006']).sum()/1e9
                ref_rev = (dfp0['ref_combined'] * dfp0['s006']).sum()/1e9
            if segmentchoice=='pn':
                base_rev = (dfpn['combined'] * dfpn['s006']).sum()/1e9
                ref_rev = (dfpn['ref_combined'] * dfpn['s006']).sum()/1e9
            if segmentchoice=='0p':
                base_rev = (df0p['combined'] * df0p['s006']).sum()/1e9
                ref_rev = (df0p['ref_combined'] * df0p['s006']).sum()/1e9
            if segmentchoice=='00':
                base_rev = (df00['combined'] * df00['s006']).sum()/1e9
                ref_rev = (df00['ref_combined'] * df00['s006']).sum()/1e9
            if segmentchoice=='0n':
                base_rev = (df0n['combined'] * df0n['s006']).sum()/1e9
                ref_rev = (df0n['ref_combined'] * df0n['s006']).sum()/1e9
            if segmentchoice=='np':
                base_rev = (dfnp['combined'] * dfnp['s006']).sum()/1e9
                ref_rev = (dfnp['ref_combined'] * dfnp['s006']).sum()/1e9
            if segmentchoice=='n0':
                base_rev = (dfn0['combined'] * dfn0['s006']).sum()/1e9
                ref_rev = (dfn0['ref_combined'] * dfn0['s006']).sum()/1e9
            if segmentchoice=='nn':
                base_rev = (dfnn['combined'] * dfnn['s006']).sum()/1e9
                ref_rev = (dfnn['ref_combined'] * dfnn['s006']).sum()/1e9
            if segmentchoice=='total':
                base_rev= (base_df['combined'] * base_df['s006']).sum()/1e9
                ref_rev = (ref_df['combined'] * ref_df['s006']).sum()/1e9
            
            dif_rev = ref_rev - base_rev
            col.append(round(dif_rev,2))
        df[year] = col

    # Revenue Table
    table = df
    return table

In [ ]:
# Non-Refundable Portion by Liab

def nonref(refchoice, basechoice, baselinelaw, segmentchoice, subchoice, cgchoice):
    rec = Records()
    refs_policy = refchoice
    refs_base = basechoice
    df = pd.DataFrame(index=["Non-Refundable Portion of Credits"
                         ])
    
    for year in range(2021,2022):
        pol_base = Policy()
        pol_base.implement_reform(baselinelaw)
        pol_ref = Policy()
        pol_ref.implement_reform(baselinelaw)
        col=[]
        response_elasticities = {'sub': subchoice, 'cg': cgchoice}
        for (ref_policy, ref_base) in zip(refs_policy, refs_base):
            pol_base.implement_reform(ref_base)
            calc_base=Calculator(pol_base, rec)
            calc_base.advance_to_year(year)
            pol_ref.implement_reform(ref_policy)
            calc_ref=Calculator(pol_ref, rec)
            calc_ref.advance_to_year(year)
            calc_ref.calc_all()
            base_df, ref_df = response(calc_base, calc_ref, response_elasticities)

            df1 = copy.deepcopy(base_df)
            df1 = df1.assign(ref_iitax = ref_df['iitax'])
            df1 = df1.assign(ref_c07100 = ref_df['c07100'])

            dfp = df1[df1['iitax']>0]
            df0 = df1[df1['iitax']==0]
            dfn = df1[df1['iitax']<0]

            dfpp = dfp[dfp['ref_iitax']>0]
            dfp0 = dfp[dfp['ref_iitax']==0]
            dfpn = dfp[dfp['ref_iitax']<0]

            df0p = df0[df0['ref_iitax']>0]
            df00 = df0[df0['ref_iitax']==0]
            df0n = df0[df0['ref_iitax']<0]

            dfnp = dfn[dfn['ref_iitax']>0]
            dfn0 = dfn[dfn['ref_iitax']==0]
            dfnn = dfn[dfn['ref_iitax']<0]
            
            if segmentchoice=='pp':
                base_rev = (dfpp['c07100'] * dfpp['s006']).sum()/1e9
                ref_rev = (dfpp['ref_c07100'] * dfpp['s006']).sum()/1e9
            if segmentchoice=='p0':
                base_rev = (dfp0['c07100'] * dfp0['s006']).sum()/1e9
                ref_rev = (dfp0['ref_c07100'] * dfp0['s006']).sum()/1e9
            if segmentchoice=='pn':
                base_rev = (dfpn['c07100'] * dfpn['s006']).sum()/1e9
                ref_rev = (dfpn['ref_c07100'] * dfpn['s006']).sum()/1e9
            if segmentchoice=='0p':
                base_rev = (df0p['c07100'] * df0p['s006']).sum()/1e9
                ref_rev = (df0p['ref_c07100'] * df0p['s006']).sum()/1e9
            if segmentchoice=='00':
                base_rev = (df00['c07100'] * df00['s006']).sum()/1e9
                ref_rev = (df00['ref_c07100'] * df00['s006']).sum()/1e9
            if segmentchoice=='0n':
                base_rev = (df0n['c07100'] * df0n['s006']).sum()/1e9
                ref_rev = (df0n['ref_c07100'] * df0n['s006']).sum()/1e9
            if segmentchoice=='np':
                base_rev = (dfnp['c07100'] * dfnp['s006']).sum()/1e9
                ref_rev = (dfnp['ref_c07100'] * dfnp['s006']).sum()/1e9
            if segmentchoice=='n0':
                base_rev = (dfn0['c07100'] * dfn0['s006']).sum()/1e9
                ref_rev = (dfn0['ref_c07100'] * dfn0['s006']).sum()/1e9
            if segmentchoice=='nn':
                base_rev = (dfnn['c07100'] * dfnn['s006']).sum()/1e9
                ref_rev = (dfnn['ref_c07100'] * dfnn['s006']).sum()/1e9
            if segmentchoice=='total':
                base_rev= (base_df['c07100'] * base_df['s006']).sum()/1e9
                ref_rev = (ref_df['c07100'] * ref_df['s006']).sum()/1e9

            dif_rev = ref_rev - base_rev
            col.append(round(dif_rev,2))
        df[year] = col

    # Revenue Table
    table = df
    return table

In [ ]:
# Refund Eligible Portion by Liab

def ref(refchoice, basechoice, baselinelaw, segmentchoice, subchoice, cgchoice):
    rec = Records()
    refs_policy = refchoice
    refs_base = basechoice
    df = pd.DataFrame(index=["Refundable Portion of Credits"
                         ])

    for year in range(2021,2022):
        pol_base = Policy()
        pol_base.implement_reform(baselinelaw)
        pol_ref = Policy()
        pol_ref.implement_reform(baselinelaw)
        col=[]
        response_elasticities = {'sub': subchoice, 'cg': cgchoice}
        for (ref_policy, ref_base) in zip(refs_policy, refs_base):
            pol_base.implement_reform(ref_base)
            calc_base=Calculator(pol_base, rec)
            calc_base.advance_to_year(year)
            pol_ref.implement_reform(ref_policy)
            calc_ref=Calculator(pol_ref, rec)
            calc_ref.advance_to_year(year)
            calc_ref.calc_all()
            base_df, ref_df = response(calc_base, calc_ref, response_elasticities)

            df1 = copy.deepcopy(base_df)
            df1 = df1.assign(ref_iitax = ref_df['iitax'])
            df1 = df1.assign(ref_refund = ref_df['refund'])

            dfp = df1[df1['iitax']>0]
            df0 = df1[df1['iitax']==0]
            dfn = df1[df1['iitax']<0]

            dfpp = dfp[dfp['ref_iitax']>0]
            dfp0 = dfp[dfp['ref_iitax']==0]
            dfpn = dfp[dfp['ref_iitax']<0]

            df0p = df0[df0['ref_iitax']>0]
            df00 = df0[df0['ref_iitax']==0]
            df0n = df0[df0['ref_iitax']<0]

            dfnp = dfn[dfn['ref_iitax']>0]
            dfn0 = dfn[dfn['ref_iitax']==0]
            dfnn = dfn[dfn['ref_iitax']<0]

            if segmentchoice=='pp':
                base_rev = (dfpp['refund'] * dfpp['s006']).sum()/1e9
                ref_rev = (dfpp['ref_refund'] * dfpp['s006']).sum()/1e9
            if segmentchoice=='p0':
                base_rev = (dfp0['refund'] * dfp0['s006']).sum()/1e9
                ref_rev = (dfp0['ref_refund'] * dfp0['s006']).sum()/1e9
            if segmentchoice=='pn':
                base_rev = (dfpn['refund'] * dfpn['s006']).sum()/1e9
                ref_rev = (dfpn['ref_refund'] * dfpn['s006']).sum()/1e9
            if segmentchoice=='0p':
                base_rev = (df0p['refund'] * df0p['s006']).sum()/1e9
                ref_rev = (df0p['ref_refund'] * df0p['s006']).sum()/1e9
            if segmentchoice=='00':
                base_rev = (df00['refund'] * df00['s006']).sum()/1e9
                ref_rev = (df00['ref_refund'] * df00['s006']).sum()/1e9
            if segmentchoice=='0n':
                base_rev = (df0n['refund'] * df0n['s006']).sum()/1e9
                ref_rev = (df0n['ref_refund'] * df0n['s006']).sum()/1e9
            if segmentchoice=='np':
                base_rev = (dfnp['refund'] * dfnp['s006']).sum()/1e9
                ref_rev = (dfnp['ref_refund'] * dfnp['s006']).sum()/1e9
            if segmentchoice=='n0':
                base_rev = (dfn0['refund'] * dfn0['s006']).sum()/1e9
                ref_rev = (dfn0['ref_refund'] * dfn0['s006']).sum()/1e9
            if segmentchoice=='nn':
                base_rev = (dfnn['refund'] * dfnn['s006']).sum()/1e9
                ref_rev = (dfnn['ref_refund'] * dfnn['s006']).sum()/1e9
            if segmentchoice=='total':
                base_rev= (base_df['refund'] * base_df['s006']).sum()/1e9
                ref_rev = (ref_df['refund'] * ref_df['s006']).sum()/1e9

            dif_rev = ref_rev - base_rev
            col.append(round(dif_rev,2))
        df[year] = col

    # Revenue Table
    table = df
    return table

In [ ]:
# All Family Benefits Tax Liab by Liab

def liab(refchoice, basechoice, baselinelaw, segmentchoice, subchoice, cgchoice):
    rec = Records()
    refs_policy = refchoice
    refs_base = basechoice
    df = pd.DataFrame(index=["Change in Tax Liability"
                         ])

    for year in range(2021,2022):
        pol_base = Policy()
        pol_base.implement_reform(baselinelaw)
        pol_ref = Policy()
        pol_ref.implement_reform(baselinelaw)
        col=[]
        response_elasticities = {'sub': subchoice, 'cg': cgchoice}
        for (ref_policy, ref_base) in zip(refs_policy, refs_base):
            pol_base.implement_reform(ref_base)
            calc_base=Calculator(pol_base, rec)
            calc_base.advance_to_year(year)
            pol_ref.implement_reform(ref_policy)
            calc_ref=Calculator(pol_ref, rec)
            calc_ref.advance_to_year(year)
            calc_ref.calc_all()
            base_df, ref_df = response(calc_base, calc_ref, response_elasticities)

            df1 = copy.deepcopy(base_df)
            df1 = df1.assign(ref_iitax = ref_df['iitax'])
            df1 = df1.assign(ref_c05800 = ref_df['c05800'])


            dfp = df1[df1['iitax']>0]
            df0 = df1[df1['iitax']==0]
            dfn = df1[df1['iitax']<0]

            dfpp = dfp[dfp['ref_iitax']>0]
            dfp0 = dfp[dfp['ref_iitax']==0]
            dfpn = dfp[dfp['ref_iitax']<0]

            df0p = df0[df0['ref_iitax']>0]
            df00 = df0[df0['ref_iitax']==0]
            df0n = df0[df0['ref_iitax']<0]

            dfnp = dfn[dfn['ref_iitax']>0]
            dfn0 = dfn[dfn['ref_iitax']==0]
            dfnn = dfn[dfn['ref_iitax']<0]

            if segmentchoice=='pp':
                base_rev = (dfpp['c05800'] * dfpp['s006']).sum()/1e9
                ref_rev = (dfpp['ref_c05800'] * dfpp['s006']).sum()/1e9
            if segmentchoice=='p0':
                base_rev = (dfp0['c05800'] * dfp0['s006']).sum()/1e9
                ref_rev = (dfp0['ref_c05800'] * dfp0['s006']).sum()/1e9
            if segmentchoice=='pn':
                base_rev = (dfpn['c05800'] * dfpn['s006']).sum()/1e9
                ref_rev = (dfpn['ref_c05800'] * dfpn['s006']).sum()/1e9
            if segmentchoice=='0p':
                base_rev = (df0p['c05800'] * df0p['s006']).sum()/1e9
                ref_rev = (df0p['ref_c05800'] * df0p['s006']).sum()/1e9
            if segmentchoice=='00':
                base_rev = (df00['c05800'] * df00['s006']).sum()/1e9
                ref_rev = (df00['ref_c05800'] * df00['s006']).sum()/1e9
            if segmentchoice=='0n':
                base_rev = (df0n['c05800'] * df0n['s006']).sum()/1e9
                ref_rev = (df0n['ref_c05800'] * df0n['s006']).sum()/1e9
            if segmentchoice=='np':
                base_rev = (dfnp['c05800'] * dfnp['s006']).sum()/1e9
                ref_rev = (dfnp['ref_c05800'] * dfnp['s006']).sum()/1e9
            if segmentchoice=='n0':
                base_rev = (dfn0['c05800'] * dfn0['s006']).sum()/1e9
                ref_rev = (dfn0['ref_c05800'] * dfn0['s006']).sum()/1e9
            if segmentchoice=='nn':
                base_rev = (dfnn['c05800'] * dfnn['s006']).sum()/1e9
                ref_rev = (dfnn['ref_c05800'] * dfnn['s006']).sum()/1e9
            if segmentchoice=='total':
                base_rev= (base_df['c05800'] * base_df['s006']).sum()/1e9
                ref_rev = (ref_df['c05800'] * ref_df['s006']).sum()/1e9

            dif_rev = ref_rev - base_rev
            col.append(round(dif_rev,2))
        df[year] = col

    # Revenue Table
    table = df
    return table

In [ ]:
# Baseline IITAX liability by group

def baseiitax(refchoice, basechoice, baselinelaw, segmentchoice, subchoice, cgchoice):
    rec = Records()
    refs_policy = refchoice
    refs_base = basechoice
    df = pd.DataFrame(index=["Baseline Revenue"
                         ])

    for year in range(2021,2022):
        pol_base = Policy()
        pol_base.implement_reform(baselinelaw)
        pol_ref = Policy()
        pol_ref.implement_reform(baselinelaw)
        col=[]
        response_elasticities = {'sub': subchoice, 'cg': cgchoice}
        for (ref_policy, ref_base) in zip(refs_policy, refs_base):
            pol_base.implement_reform(ref_base)
            calc_base=Calculator(pol_base, rec)
            calc_base.advance_to_year(year)
            pol_ref.implement_reform(ref_policy)
            calc_ref=Calculator(pol_ref, rec)
            calc_ref.advance_to_year(year)
            calc_ref.calc_all()
            base_df, ref_df = response(calc_base, calc_ref, response_elasticities)

            df1 = copy.deepcopy(base_df)
            df1 = df1.assign(ref_iitax = ref_df['iitax'])

            dfp = df1[df1['iitax']>0]
            df0 = df1[df1['iitax']==0]
            dfn = df1[df1['iitax']<0]

            dfpp = dfp[dfp['ref_iitax']>0]
            dfp0 = dfp[dfp['ref_iitax']==0]
            dfpn = dfp[dfp['ref_iitax']<0]

            df0p = df0[df0['ref_iitax']>0]
            df00 = df0[df0['ref_iitax']==0]
            df0n = df0[df0['ref_iitax']<0]

            dfnp = dfn[dfn['ref_iitax']>0]
            dfn0 = dfn[dfn['ref_iitax']==0]
            dfnn = dfn[dfn['ref_iitax']<0]
            
            if segmentchoice=='pp':
                base_rev = (dfpp['iitax'] * dfpp['s006']).sum()/1e9
                ref_rev = (dfpp['ref_iitax'] * dfpp['s006']).sum()/1e9
            if segmentchoice=='p0':
                base_rev = (dfp0['iitax'] * dfp0['s006']).sum()/1e9
                ref_rev = (dfp0['ref_iitax'] * dfp0['s006']).sum()/1e9
            if segmentchoice=='pn':
                base_rev = (dfpn['iitax'] * dfpn['s006']).sum()/1e9
                ref_rev = (dfpn['ref_iitax'] * dfpn['s006']).sum()/1e9
            if segmentchoice=='0p':
                base_rev = (df0p['iitax'] * df0p['s006']).sum()/1e9
                ref_rev = (df0p['ref_iitax'] * df0p['s006']).sum()/1e9
            if segmentchoice=='00':
                base_rev = (df00['iitax'] * df00['s006']).sum()/1e9
                ref_rev = (df00['ref_iitax'] * df00['s006']).sum()/1e9
            if segmentchoice=='0n':
                base_rev = (df0n['iitax'] * df0n['s006']).sum()/1e9
                ref_rev = (df0n['ref_iitax'] * df0n['s006']).sum()/1e9
            if segmentchoice=='np':
                base_rev = (dfnp['iitax'] * dfnp['s006']).sum()/1e9
                ref_rev = (dfnp['ref_iitax'] * dfnp['s006']).sum()/1e9
            if segmentchoice=='n0':
                base_rev = (dfn0['iitax'] * dfn0['s006']).sum()/1e9
                ref_rev = (dfn0['ref_iitax'] * dfn0['s006']).sum()/1e9
            if segmentchoice=='nn':
                base_rev = (dfnn['iitax'] * dfnn['s006']).sum()/1e9
                ref_rev = (dfnn['ref_iitax'] * dfnn['s006']).sum()/1e9
            if segmentchoice=='total':
                base_rev= (base_df['iitax'] * base_df['s006']).sum()/1e9
                ref_rev = (ref_df['iitax'] * ref_df['s006']).sum()/1e9
                
            dif_rev = base_rev
            col.append(round(dif_rev,2))
        df[year] = col

    # Revenue Table
    table = df
    return table

In [ ]:
# Begin Model Run

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reformClinton]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_Clinton = [df1,df4,df5]

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reformBush1]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_Bush1 = [df1,df4,df5]

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reformBush2]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_Bush2 = [df1,df4,df5]

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reformBush3]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_Bush3 = [df1,df4,df5]

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reform2008]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_2008 = [df1,df4,df5]

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reformOB]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_OB = [df1,df4,df5]

In [ ]:
######################
sub = 0 
cg =  0 
reform = [reformTCJA]
baseline = [reform0]
baselinelaw = reform0
######################

# Value of All Family Benefits
df1 = combined(reform, baseline, baselinelaw, 'total',sub,cg)
df1.index.name = 'Value of All Family Benefits'

# Non-Refundable Portion
df4 = nonref(reform, baseline, baselinelaw,'total',sub,cg)
df4.index.name = 'Non-Refundable Portion'

# Refund Eligible Portion
df5 = ref(reform, baseline, baselinelaw,'total',sub,cg)
df5.index.name = 'Refund Eligible Portion'

# list of dataframes
dfs_TCJA = [df1,df4,df5]

In [ ]:
# Write Output

In [ ]:
writer = pd.ExcelWriter('../output/ctc-budget-estimates_FINAL.xlsx', engine='xlsxwriter')
row2=0
for dataframe in dfs_Clinton:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='1997-1999 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
row2=0
for dataframe in dfs_Bush1:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='2001-2002 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
row2=0
for dataframe in dfs_Bush2:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='2003 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
row2=0
for dataframe in dfs_Bush3:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='2004-2007 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
row2=0
for dataframe in dfs_2008:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='2008 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
row2=0
for dataframe in dfs_OB:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='2009-2017 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
row2=0
for dataframe in dfs_TCJA:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='Current Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3 
writer.save()